# Which TCRs affect a person's CMV status?

# Introduction

People’s pathogen exposure can be influenced by immunosequencing. In our project we focus on T cell repertoire and profile the T cell repertoire of 666 subjects with known cytomegalovirus (CMV) serostatus. To find out T cell receptors (TCRs) associated with CMV status, we implement two classification methods, which are beta-binomial model and naive bayes classifier. The classification methods would be also applied to predict the CMV status of a novel subject. Then leave-one-out cross validation is used to test the two methods. Also, we get LOO by stan to compare the two models.

This project is a continuation of the study _Immunosequencing identifies signatures of cytomegalovirus exposure history and HLA-mediated effects on the T cell repertoire_ by Emerson et al (2017). We use the same data, but slightly differing statistical methods, as well as different objectives.

# Methods

This project consists of four main parts: defining the significance levels of group differences in TCRs, Bayesian inference for the proportions of the TCRs in each group, creating a predictive model, as well as testing the model and comparing it to the Beta Binomial model of Emerson et al. (2017). 

## Fisher's exact test

The number of possible TCR combinations is very large. For a subsample of 50 people, the total number of unique TCRs was over one million. For this reason, it is not plausible to use every TCR combination, even though they might carry some information.

To counter this problem, the association between a given TCR and CMV status can be quantified. For this purpose, the Fisher's exact test was used. It provides an exact p-value, that denotes the probability that observed differences in proportion between two groups can be attributed to chance.

We used the same significance level that the Emerson et al. (2017) use. It had been 

## Binomial model for the distribution of TCRs

In the original paper, the TCRs that are associated with a positive CMV status were modeled as occuring with equal probability in each group. Thus, the paper only focused on the number of CMV associated, rather than the individual TCRs. We wanted to test the assumption that all TCRs have the same probability to appear within one CMV group.

Another way to model the TCRs appearing would be to denote a probability $p_{ij}$ for each $TCR_j$ to appear in a subject of CMV group $C_i$. Then the number of subjects with the given TCR would be binomially distributed. Based on this assumption, we can perform Bayesian inference and obtain the distribution of $p_{ij}$. Since the number of subjects with a TCR is binomially distributed, the likelihood of $p_{ij}$ is Beta distributed. Since we have no other data of the distribution of the TCRs, we use a uniform prior. Thus, the posterior is also a Beta distribution.


## Naive Bayes classifier based on Bayesian inference

The aim was to create a classifier based on the observations of the co-occurence of TCRs and CMV statuses. For this purpose, a Bernoulli naive Bayes classifier was used. A Bernoulli naive Bayes is good for the purpose: it uses binary data to sort samples into categories; in this case two. The model has the following structure

$ p(C_i \mid {\bf x} ) \propto p({\bf x} \mid C_i ) p(C_i) = p(C_i) \Pi _{j=1}^{N} p(x_j \mid C_i)^{b_j}(1 - p(x_j \mid C_i))^{(1-b_j)} $

where $b_j$ denotes the boolean value of having the TCR of the index $j$. 

Moreover, these probabilities are based on the expected values of the proportions in the sample. This means that $p(x_j \mid C_i)$ is actually $p(x_j \mid C_i, {\bf x}_0)$ in this case. Because they are modelled as fixed probabilities in each group, these underlying probabilities are Beta distributed. Due to the independence assumption, we can first calculate the expected values of the probabilites, and then calculate the product for class probability estimation.

The proportional values are transformed into probabilities

$p(C_i \mid {\bf x} ) = \frac{p({\bf x} \mid C_i ) p(C_i) }{ \Sigma_{n=1}^K p({\bf x} \mid C_n ) p(C_n)} $

where $K$ is the number of classes. Since $CMV^+$ and $CMV^-$ are complements, the number of classes is 2 in this case. Based on the observations ${\bf x_0}$, the estimate becomes

$p(C_i \mid {\bf x}, {\bf x_0} ) =\frac{P(T \mid CMV^+) P(CMV^+)}{P(T \mid CMV^+) + P(T \mid CMV^-)}$
$ = \frac{\Pi_{i=1}^n P(TCR_i \mid CMV^+)P(CMV^+)}{ \Pi_{i=1}^n P(TCR_i \mid CMV^+)P(CMV^+) + \Pi_{i=1}^n P(TCR_i \mid CMV^-)P(CMV^+) } $

$ = \frac{\frac{m_i^++1}{m_i^++2} \Pi_{i=1}^n \frac{l_i^++1}{m_i^++2} }{ \frac{m_i^++1}{m_i^++2}\Pi_{i=1}^n \frac{l_i^++1}{m_i^++2} + \frac{m_i^-+1}{m_i^-+2}\Pi_{i=1}^n \frac{l_i^-+1}{m_i^-+2} } $

$ = \frac{ (m_i^++1)\Pi_{i=1}^n \frac{l_i^++1}{m_i^++2} }{ (m_i^++1)\Pi_{i=1}^n \frac{l_i^++1}{m_i^++2} + (m_i^-+1)\Pi_{i=1}^n \frac{l_i^-+1}{m_i^-+2} } $


where $l^+$ and $l^-$ denote occurences of the given TCR in the CMV positive group and CMV negative group respectively. Likewise, $m^+$ and $m^-$ denote the total sizes of these groups. $T$ denotes some combinations of TCRs occuring.


## Cross validation (LOO)

In order to compare our model with the one used in the original paper, some validation method must be used. For this purpose, we chose LOO cross validation.

LOO cross validation essentially measures the predictive performance of a model for a given data point, when the model is based on all the other data.

We use two different metrics for the . Because of its intuitive nature, accuracy (how many percent of data points are classified correctly) is used. To obtain a better of the probabilistic predictions the model is making, log-loss is used as well.

The original purpose of this project was to compare the original, beta-binomial model to our multidimensional beta model. However, the original model proved to be difficult to implement, and we did not get proper results for it.

In [8]:
# maybe some python code here

# Results

The Fisher exact test produced rather straightforward results. A total of 164 TCRs had statistically significant differences in proportion between the two groups. 

The model based on individually beta distributed TCRs performed relatively well. In LOO terms, it had an accuracy of 86.2 per cent, while the average log loss was -12.153.

We were able to compile the model that Emerson et al. (2017) suggested. We were, however, not able to create a predictor for this model. Thus, the results of the comparison between these models are indecisive. Moreover, Emerson et al. (2017) did not provide their statistics for accuracy and log-loss, even though they mentioned that they used it in the evaluation of CMV associated TCRs.

In [13]:
# definitely some code here

import numpy as np
data = np.random.rand(3,2)

print(data)

[[0.65858662 0.54623998]
 [0.03014943 0.56086935]
 [0.49236616 0.33564563]]


# Discussion

Generally, the model performed well. However, many of the predicions were skewed towards either side of the probability. The model seemed to be over-confident.

A significant problem in this study was the fact that the .

Also, it is possible to also utilize TCRs with lower significance levels. The ones used in this dataset had very low p-values. LOO validation provides a good framework for testing the rigidity of 

In [14]:
# probably no or little code

# References

- Emerson, Ryan O., et al. "Immunosequencing identifies signatures of cytomegalovirus exposure history and HLA-mediated effects on the T cell repertoire." Nature genetics 49.5 (2017): 659.

# Appendix

Maybe stan or even python code. Let's see if it's better to have it in the actual report sections or here. Perhaps include some of the code in the report and full code here.

In [6]:
# python code

import numpy as np
from math import log
import math

from numpy import genfromtxt

my_data = genfromtxt('supplementaryTables/SUPP.csv', delimiter=';')

n_plus = 641 - 365
n_minus = 365

cmv_plus = my_data[:, 3] + 1
cmv_minus = my_data[:, 4] + 1

cmv_plus = cmv_plus[1:]
cmv_minus = cmv_minus[1:]


tcrs = []

f = open('supplementaryTables/SUPP2.csv', "r")

for line in f.readlines():
	tcrs.append(line.rstrip())

# binary data
# each row is a person
# the last number in the row is CMV class
# all other data is TCR's

# generate data with the following probabilities to have a given TCR
# the last one denotes class, thus 1.0 and 0.0

def classify(filename):
	status = True
	if "negative" in filename:
		status = False

	f = open(filename, "r")

	indices = []
	i = 0
	for line in f.readlines():
		l = line.rstrip()
		if i > 0 and len(l) > 0:
			index = tcrs.index(l)
			indices.append(tcrs.index(l))
		i += 1

	contr = np.zeros(cmv_plus.shape)
	for index in indices:
		contr[index] = 1.0

	# calculate probabilities without the given data point
	probs_1 = cmv_plus / (n_plus + 2)
	probs_0 = (cmv_minus-contr) / (n_minus + 2)

	if status:
		probs_1 = (cmv_plus-contr) / (n_plus + 2)
		probs_0 = cmv_minus / (n_minus + 2)

	probs_1 = np.abs(np.abs(probs_1 - contr) - 1)
	probs_0 = np.abs(np.abs(probs_0 - contr) - 1)

	# calculate the likelihoods
	p_a = np.prod(probs_1) * (n_plus + 1) / (n_plus + 2)
	p_b = np.prod(probs_0) * (n_minus + 1) / (n_minus + 2)

	# calculate class probabilities based on the likelihoods
	p_total = p_a / (p_a + p_b)

	# return more likely class and its respective probability 
	c = 1
	if p_b > p_a:
		c = 0
		p_total = 1 - p_total

	return c, p_total


def loo(files):
	logloss = 0.0
	corr = 0
	tot = 0
	negs = 0
	for file in files:

		if "positive" in file or "negative" in file:
			tot += 1
			correct_class = 1
			if "negative" in file:
				correct_class = 0
				negs += 1
			c, p = classify(file)

			# prevent numerical errors in logloss
			if p < 0.0000000001:
				p = 0.0000000001
			elif p > 1.0 - 0.0000000001:
				p = 1.0 - 0.0000000001
			if c == correct_class:
				corr += 1
				logloss += log(p)
			logloss += log(1 - p)

	print("Accuracy", corr / tot)
	logloss = logloss / tot
	print("Log loss", logloss)
	print("total subjects", tot, "CMV negative", negs)

import glob
fs = glob.glob("parsed3/*.csv")

loo(fs)

Accuracy 0.8627145085803433
Log loss -12.153544163581788
total subjects 641 CMV negative 365


In [11]:
#// stan code